## 기본 합성곱 신경망 구현 (Vanilla CNN)

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
epochs = 10

### Model

In [3]:
class ConvNet(tf.keras.Model):
    def __init__(self):
        super(ConvNet, self).__init__()
        conv2d = tf.keras.layers.Conv2D
        maxpool = tf.keras.layers.MaxPool2D
        flatten = tf.keras.layers.Flatten
        dense = tf.keras.layers.Dense
        
        ## VGG-19 의 형태
        self.sequence = []
        self.sequence.append(conv2d(16, (3, 3), padding='same', activation='relu'))   # 28 x 28 x 16
        self.sequence.append(conv2d(16, (3, 3), padding='same', activation='relu'))
        self.sequence.append(maxpool((2, 2)))                                         # 14 x 14 x 16
        
        self.sequence.append(conv2d(32, (3, 3), padding='same', activation='relu'))   # 14 x 14 x 32
        self.sequence.append(conv2d(32, (3, 3), padding='same', activation='relu'))
        self.sequence.append(maxpool((2, 2)))                                         # 7 x 7 x 32
        
        self.sequence.append(conv2d(64, (3, 3), padding='same', activation='relu'))   # 7 x 7 x 64
        self.sequence.append(conv2d(64, (3, 3), padding='same', activation='relu'))
        self.sequence.append(flatten())                                                 # 7 x 7 x 64 = 3136
        
        self.sequence.append(dense(2048, activation='relu'))
        self.sequence.append(dense(10, activation='softmax'))
    
    def call(self, x, training=False, mask=None):
        for layer in self.sequence:
            x = layer(x)
        return x

### 학습, 테스트 루프

In [4]:
@tf.function
def train_step(model, inputs, labels, loss_object, optimizer, train_loss, train_accuracy):
    with tf.GradientTape() as tape:
        predictions = model(inputs)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)   # loss : scalar 를 trainable_variable로 미분해준 값(벡터)
    
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))   # gradients와 그에 해당하는 t_v를 같이 넣어 학습
    train_loss(loss)   # 손실들을 종합
    train_accuracy(labels, predictions)   # 평가지표로 계산
    
@tf.function
def test_step(model, inputs, labels, loss_object, test_loss, test_accuracy):
    predictions = model(inputs)
    t_loss = loss_object(labels, predictions)
    
    test_loss(t_loss)   # 손실들을 종합
    test_accuracy(labels, predictions)   # 평가지표로 계산

### Dataset [mnist]

In [5]:
mnist = tf.keras.datasets.mnist

(train_x, train_y), (test_x, test_y) = mnist.load_data()

train_x, test_x = train_x / 255., test_x / 255.
train_x, test_x = train_x[..., tf.newaxis].astype(np.float32), test_x[..., tf.newaxis].astype(np.float32)

train_ds = tf.data.Dataset.from_tensor_slices((train_x, train_y)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((test_x, test_y)).batch(32)

### 학습 환경

- 모델 생성, 손실함수, 최적화 알고리즘, 평가지표 정의

In [6]:
model = ConvNet()

loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

### 학습

In [7]:
for epoch in range(epochs):
    for images, labels in train_ds:
        train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy)
        
    for test_images, test_labels in test_ds:
        test_step(model, test_images, test_labels, loss_object, test_loss, test_accuracy)
        
    template = 'Epoch {0:<2}, Loss : {1:.4f}, Accuracy : {2:.4f}, Test Loss : {3:.4f}, Test Accuracy : {4:.4f}'
    print(template.format(epoch + 1, 
                         train_loss.result(), 
                         train_accuracy.result() * 100, 
                         test_loss.result(), 
                         test_accuracy.result() * 100))

Epoch 1, Loss : 0.1144, Accuracy : 96.3817, Test Loss : 0.0359, Test Accuracy : 98.9500
Epoch 2, Loss : 0.0802, Accuracy : 97.4850, Test Loss : 0.0377, Test Accuracy : 98.9100
Epoch 3, Loss : 0.0649, Accuracy : 97.9933, Test Loss : 0.0435, Test Accuracy : 98.8333
Epoch 4, Loss : 0.0556, Accuracy : 98.2950, Test Loss : 0.0404, Test Accuracy : 98.8750
Epoch 5, Loss : 0.0491, Accuracy : 98.4930, Test Loss : 0.0383, Test Accuracy : 98.9320
Epoch 6, Loss : 0.0443, Accuracy : 98.6392, Test Loss : 0.0385, Test Accuracy : 98.9433
Epoch 7, Loss : 0.0405, Accuracy : 98.7519, Test Loss : 0.0387, Test Accuracy : 98.9629
Epoch 8, Loss : 0.0373, Accuracy : 98.8521, Test Loss : 0.0386, Test Accuracy : 98.9700
Epoch 9, Loss : 0.0351, Accuracy : 98.9217, Test Loss : 0.0386, Test Accuracy : 98.9633
Epoch 10, Loss : 0.0331, Accuracy : 98.9853, Test Loss : 0.0391, Test Accuracy : 98.9690


---

---

## Residual Network (ResNet) 구현

ResNet <- Residual Layer <- Residual Unit

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
epochs = 10

### Residual Unit

<img src='https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FbzW8VQ%2FbtqF63hMf2h%2FXZ6zUpZ6Bu2veeMJdkcvL0%2Fimg.png' width=300>

In [3]:
class ResidualUnit(tf.keras.Model):
    def __init__(self, filter_in, filter_out, kernel_size):
        super(ResidualUnit, self).__init__()
        ## Preactivation
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.conv1 = tf.keras.layers.Conv2D(filter_out, kernel_size, padding='same')
        
        self.bn2 = tf.keras.layers.BatchNormalization()
        self.conv2 = tf.keras.layers.Conv2D(filter_out, kernel_size, padding='same')
        
        ## bn - relu - conv 를 거친 값과 입력을 더해주는 skip connection 에서 input과 output의 채널이 같아야 더해지기 때문에 추가해주는 부분.
        if filter_in == filter_out:
            self.identity = lambda x: x
        else:
            self.identity = tf.keras.layers.Conv2D(filter_out, (1, 1), padding='same')
    
    def call(self, x, training=False, mask=None):
        h = self.bn1(x, training=training)   # batch normalization은 training과 test일때 달라야하기 때문에
        h = tf.nn.relu(h)
        h = self.conv1(h)
        
        h = self.bn2(h, training=training)
        h = tf.nn.relu(h)
        h = self.conv2(h)
        
        return self.identity(x) + h

### Residual Layer

In [10]:
class ResnetLayer(tf.keras.Model):
    def __init__(self, filter_in, filters, kernel_size):
        super(ResnetLayer, self).__init__()
        self.sequence = []
        for f_in, f_out in zip([filter_in] + list(filters), filters):
            self.sequence.append(ResidualUnit(f_in, f_out, kernel_size))
    
    def call(self, x, training=False, mask=None):
        for unit in self.sequence:
            x = unit(x, training=training)
        return x

### ResNet

In [11]:
class ResNet(tf.keras.Model):
    def __init__(self):
        super(ResNet, self).__init__()
        # 처음에 feature를 하나 뽑아놔야 더 효과적이다.
        self.conv1 = tf.keras.layers.Conv2D(8, (3, 3), padding='same', activation='relu')   # 28 x 28 x 8
        
        self.res1 = ResnetLayer(8, (16, 16), (3, 3))                                        # 28 x 28 x 16
        self.pool1 = tf.keras.layers.MaxPool2D((2, 2))                                      # 14 x 14 x 16
        
        self.res2 = ResnetLayer(16, (32, 32), (3, 3))                                        # 14 x 14 x 32
        self.pool2 = tf.keras.layers.MaxPool2D((2, 2))                                      # 7 x 7 x 32
        
        self.res3 = ResnetLayer(64, (32, 32), (3, 3))                                        # 7 x 7 x 64
        
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(128, activation='relu')
        self.dense2 = tf.keras.layers.Dense(10, activation='softmax')
        
    def call(self, x, training=False, mask=None):
        x = self.conv1(x)
        
        x = self.res1(x, training=training)
        x = self.pool1(x)
        x = self.res2(x, training=training)
        x = self.pool2(x)
        x = self.res3(x, training=training)
        
        x = self.flatten(x)
        x = self.dense1(x)
        
        return self.dense2(x)

### Dataset [mnist]

In [12]:
mnist = tf.keras.datasets.mnist

(train_x, train_y), (test_x, test_y) = mnist.load_data()

train_x, test_x = train_x / 255., test_x / 255.
train_x, test_x = train_x[..., tf.newaxis].astype(np.float32), test_x[..., tf.newaxis].astype(np.float32)

train_ds = tf.data.Dataset.from_tensor_slices((train_x, train_y)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((test_x, test_y)).batch(32)

### 학습 환경

- 모델 생성, 손실함수, 최적화 알고리즘, 평가지표 정의

In [13]:
model = ResNet()

loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

### 학습, 테스트 루프

In [14]:
@tf.function
def train_step(model, inputs, labels, loss_object, optimizer, train_loss, train_accuracy):
    with tf.GradientTape() as tape:
        predictions = model(inputs, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)   # loss : scalar 를 trainable_variable로 미분해준 값(벡터)
    
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))   # gradients와 그에 해당하는 t_v를 같이 넣어 학습
    train_loss(loss)   # 손실들을 종합
    train_accuracy(labels, predictions)   # 평가지표로 계산
    
@tf.function
def test_step(model, inputs, labels, loss_object, test_loss, test_accuracy):
    predictions = model(inputs, training=False)
    t_loss = loss_object(labels, predictions)
    
    test_loss(t_loss)   # 손실들을 종합
    test_accuracy(labels, predictions)   # 평가지표로 계산

### 학습

In [15]:
for epoch in range(epochs):
    for images, labels in train_ds:
        train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy)
        
    for test_images, test_labels in test_ds:
        test_step(model, test_images, test_labels, loss_object, test_loss, test_accuracy)
        
    template = 'Epoch {0:<2}, Loss : {1:.4f}, Accuracy : {2:.4f}, Test Loss : {3:.4f}, Test Accuracy : {4:.4f}'
    print(template.format(epoch + 1, 
                         train_loss.result(), 
                         train_accuracy.result() * 100, 
                         test_loss.result(), 
                         test_accuracy.result() * 100))

Epoch 1, Loss : 0.1230, Accuracy : 96.4917, Test Loss : 0.1387, Test Accuracy : 95.8800
Epoch 2, Loss : 0.0912, Accuracy : 97.4325, Test Loss : 0.0980, Test Accuracy : 97.1100
Epoch 3, Loss : 0.0756, Accuracy : 97.8650, Test Loss : 0.0848, Test Accuracy : 97.5533
Epoch 4, Loss : 0.0659, Accuracy : 98.1363, Test Loss : 0.0884, Test Accuracy : 97.5875
Epoch 5, Loss : 0.0590, Accuracy : 98.3157, Test Loss : 0.0788, Test Accuracy : 97.8400
Epoch 6, Loss : 0.0535, Accuracy : 98.4703, Test Loss : 0.0768, Test Accuracy : 97.8850
Epoch 7, Loss : 0.0494, Accuracy : 98.5871, Test Loss : 0.0707, Test Accuracy : 98.0786
Epoch 8, Loss : 0.0458, Accuracy : 98.6896, Test Loss : 0.0660, Test Accuracy : 98.1900
Epoch 9, Loss : 0.0427, Accuracy : 98.7781, Test Loss : 0.0632, Test Accuracy : 98.2733
Epoch 10, Loss : 0.0402, Accuracy : 98.8482, Test Loss : 0.0594, Test Accuracy : 98.3800


---
---

## DenseNet 구현

In [18]:
import tensorflow as tf
import numpy as np

In [19]:
epochs = 10

### DenseUnit 

<img src='https://hoya012.github.io/assets/img/densenet/1_comment.png' width=500>

In [20]:
class DenseUnit(tf.keras.Model):
    def __init__(self, filter_output, kernel_size):
        super(DenseUnit, self).__init__()
        self.bn = tf.keras.layers.BatchNormalization()
        self.conv = tf.keras.layers.Conv2D(filter_output, kernel_size, padding='same')
        self.concat = tf.keras.layers.Concatenate()
    
    def call(self, x, training=False, mask=None):     # x : (Batch, H, W, Ch_in)
        h = self.bn(x, training=training)
        h = tf.nn.relu(h)
        h = self.conv(h)                              # h : (Batch, H, W, filter_output)
        return self.concat([x, h])                    #     (Batch, H, W, (Ch_in + filter_output))

### DenseLayer

In [23]:
class DenseLayer(tf.keras.Model):
    def __init__(self, num_unit, growth_rate, kernel_size):
        super(DenseLayer, self).__init__()
        self.sequence = []
        for idx in range(num_unit):
            self.sequence.append(DenseUnit(growth_rate, kernel_size))
    
    def call(self, x, training=False, mask=None):
        for unit in self.sequence:
            x = unit(x, training=training)
        return x

### Transition Layer

In [24]:
class TransitionLayer(tf.keras.Model):
    def __init__(self, filters, kernel_size):
        super(TransitionLayer, self).__init__()
        self.conv = tf.keras.layers.Conv2D(filters, kernel_size, padding='same')
        self.pool = tf.keras.layers.MaxPool2D()
        
    def call(self, x, training=False, mask=None):
        x = self.conv(x)
        return self.pool(x)

### DenseNet

In [25]:
class DenseNet(tf.keras.Model):
    def __init__(self):
        super(DenseNet, self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(8, (3, 3), padding='same', activation='relu')    # 28 x 28 x 8
        
        self.dl1 = DenseLayer(2, 4, (3, 3))                                                  # 28 x 28 x (8 + 4x2)
        self.tf1 = TransitionLayer(16, (3, 3))                                               # 14 x 14 x 16
        
        self.dl2 = DenseLayer(2, 8, (3, 3))                                                  # 14 x 14 x (16 + 8x2)
        self.tf2 = TransitionLayer(32, (3, 3))                                               # 7 x 7 x 32
        
        self.dl3 = DenseLayer(2, 16, (3, 3))                                                  # 7 x 7 x (32 + 16x2)
        
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(128, activation='relu')
        self.dense2 = tf.keras.layers.Dense(10, activation='softmax')
        
    def call(self, x, training=False, mask=None):
        h = self.conv1(x)
        
        h = self.dl1(h, training=training)
        h = self.tr1(h)
        
        h = self.dl2(h, training=training)
        h = self.tr2(h)
        
        h = self.dl3(h, training=training)
        
        h = self.flatten(h)
        h = self.dense1(h)
        return self.dense2(h)

### Dataset [mnist]

In [26]:
mnist = tf.keras.datasets.mnist

(train_x, train_y), (test_x, test_y) = mnist.load_data()

train_x, test_x = train_x / 255., test_x / 255.
train_x, test_x = train_x[..., tf.newaxis].astype(np.float32), test_x[..., tf.newaxis].astype(np.float32)

train_ds = tf.data.Dataset.from_tensor_slices((train_x, train_y)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((test_x, test_y)).batch(32)

### 학습 환경

- 모델 생성, 손실함수, 최적화 알고리즘, 평가지표 정의

In [27]:
model = ResNet()

loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

### 학습, 테스트 루프

In [28]:
@tf.function
def train_step(model, inputs, labels, loss_object, optimizer, train_loss, train_accuracy):
    with tf.GradientTape() as tape:
        predictions = model(inputs, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)   # loss : scalar 를 trainable_variable로 미분해준 값(벡터)
    
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))   # gradients와 그에 해당하는 t_v를 같이 넣어 학습
    train_loss(loss)   # 손실들을 종합
    train_accuracy(labels, predictions)   # 평가지표로 계산
    
@tf.function
def test_step(model, inputs, labels, loss_object, test_loss, test_accuracy):
    predictions = model(inputs, training=False)
    t_loss = loss_object(labels, predictions)
    
    test_loss(t_loss)   # 손실들을 종합
    test_accuracy(labels, predictions)   # 평가지표로 계산

### 학습

In [29]:
for epoch in range(epochs):
    for images, labels in train_ds:
        train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy)
        
    for test_images, test_labels in test_ds:
        test_step(model, test_images, test_labels, loss_object, test_loss, test_accuracy)
        
    template = 'Epoch {0:<2}, Loss : {1:.4f}, Accuracy : {2:.4f}, Test Loss : {3:.4f}, Test Accuracy : {4:.4f}'
    print(template.format(epoch + 1, 
                         train_loss.result(), 
                         train_accuracy.result() * 100, 
                         test_loss.result(), 
                         test_accuracy.result() * 100))

Epoch 1 , Loss : 0.1300, Accuracy : 96.2300, Test Loss : 0.0564, Test Accuracy : 98.3400
Epoch 2 , Loss : 0.0950, Accuracy : 97.2775, Test Loss : 0.0493, Test Accuracy : 98.6050
Epoch 3 , Loss : 0.0787, Accuracy : 97.7339, Test Loss : 0.0486, Test Accuracy : 98.5933
Epoch 4 , Loss : 0.0680, Accuracy : 98.0404, Test Loss : 0.0461, Test Accuracy : 98.6600
Epoch 5 , Loss : 0.0609, Accuracy : 98.2387, Test Loss : 0.0455, Test Accuracy : 98.6640
Epoch 6 , Loss : 0.0556, Accuracy : 98.3839, Test Loss : 0.0460, Test Accuracy : 98.6533
Epoch 7 , Loss : 0.0508, Accuracy : 98.5233, Test Loss : 0.0438, Test Accuracy : 98.7200
Epoch 8 , Loss : 0.0472, Accuracy : 98.6267, Test Loss : 0.0433, Test Accuracy : 98.7550
Epoch 9 , Loss : 0.0442, Accuracy : 98.7137, Test Loss : 0.0412, Test Accuracy : 98.8178
Epoch 10, Loss : 0.0415, Accuracy : 98.7953, Test Loss : 0.0425, Test Accuracy : 98.7990
